In [ ]:
# Universal Google Drive Integration (works in both Kaggle and Colab)
!pip install -q pydrive2 kaggle

import os
import json
import sys
import platform

# Detect environment
def detect_environment():
    """Detect whether we're running in Kaggle or Colab"""
    if 'google.colab' in sys.modules:
        return 'colab'
    elif 'kaggle_web_client' in sys.modules or 'kaggle' in sys.modules:
        return 'kaggle'
    else:
        return 'unknown'

env = detect_environment()
print(f"Detected environment: {env}")

# Setup Google Drive integration based on environment
if env == 'colab':
    # Colab-specific Google Drive integration
    from google.colab import drive
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from oauth2client.client import GoogleCredentials
    
    print("Mounting Google Drive in Colab environment...")
    drive.mount('/content/drive')
    
    def setup_pydrive():
        """Setup PyDrive with Colab authentication"""
        from google.colab import auth
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        return GoogleDrive(gauth)
    
    # Set output directory to mounted Google Drive
    output_dir = '/content/drive/MyDrive/Numerai_Crypto'
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to: {output_dir}")
    
elif env == 'kaggle':
    # Kaggle-specific file storage
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    print("Setting up Kaggle output directory...")
    
    # Use Kaggle's built-in output directory
    output_dir = "/kaggle/working/Numerai_Crypto"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to: {output_dir}")
    
    def setup_pydrive():
        """Setup PyDrive for Kaggle (limited functionality)"""
        try:
            # Try to use a settings.yaml file if available
            if os.path.exists('settings.yaml'):
                gauth = GoogleAuth('settings.yaml')
                if gauth.credentials is None:
                    # Try to load saved credentials
                    gauth.LoadCredentialsFile('credentials.json')
                if gauth.credentials is None:
                    print("No valid credentials found. Google Drive functionality will be limited.")
                    return None
                return GoogleDrive(gauth)
            else:
                print("No settings.yaml found. Using local storage only.")
                return None
        except Exception as e:
            print(f"Error setting up PyDrive: {e}")
            print("Google Drive functionality will be limited.")
            return None
else:
    # Default for other environments
    print("Unknown environment, using current directory for output")
    output_dir = os.path.join(os.getcwd(), "Numerai_Crypto")
    os.makedirs(output_dir, exist_ok=True)
    print(f"Files will be saved to: {output_dir}")
    
    def setup_pydrive():
        """Placeholder function for unknown environment"""
        print("Google Drive integration not available in this environment")
        return None

# Universal functions for file operations
def save_file(filename, content, is_binary=False):
    """Save a file to the appropriate location based on environment"""
    file_path = os.path.join(output_dir, filename)
    mode = 'wb' if is_binary else 'w'
    with open(file_path, mode) as f:
        f.write(content)
    print(f"Saved {filename} to {file_path}")
    return file_path

def load_file(filename, is_binary=False):
    """Load a file from the appropriate location based on environment"""
    file_path = os.path.join(output_dir, filename)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None
    
    mode = 'rb' if is_binary else 'r'
    with open(file_path, mode) as f:
        content = f.read()
    print(f"Loaded {filename} from {file_path}")
    return content

# Test the file operations
try:
    # Create a test file
    test_content = f"Test file created in {env} environment at {platform.node()}"
    test_file = "test_file.txt"
    
    # Save the test file
    save_file(test_file, test_content)
    
    # Load the test file
    loaded_content = load_file(test_file)
    
    if loaded_content == test_content:
        print("✓ File operations working correctly")
    else:
        print("✗ File operations test failed")
except Exception as e:
    print(f"Error testing file operations: {e}")

# Setup Google Drive if possible
try:
    drive = setup_pydrive()
    if drive is not None:
        print("Successfully authenticated with Google Drive")
        
        # Create Numer_crypto folder if it doesn't exist
        file_list = drive.ListFile({"q": f"title='Numer_crypto' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
        
        if len(file_list) > 0:
            # Folder exists, use its ID
            folder_id = file_list[0]["id"]
        else:
            # Create folder
            folder = drive.CreateFile({"title": "Numer_crypto", "mimeType": "application/vnd.google-apps.folder"})
            folder.Upload()
            folder_id = folder["id"]
        
        print(f"Numer_crypto folder ID: {folder_id}")
    else:
        print("Google Drive authentication not available")
        print("Using local file operations only")
except Exception as e:
    print(f"Error with Google Drive: {e}")
    print("Using local file operations only")
